<a href="https://colab.research.google.com/github/PrasadPatilPrian/PrasadPatil/blob/main/CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install keras-tuner

     |████████████████████████████████| 133 kB 33.0 MB/s 


In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [ ]:
print(tf.__version__)

2.8.0


In [ ]:
fashion_mnist=keras.datasets.fashion_mnist

In [ ]:
(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data()

4431872/4422102 [==============================] - 0s 0us/step


In [ ]:
train_images=train_images/255.0
test_images=test_images/255.0

In [ ]:
train_images[0].shape

(28, 28)

In [ ]:
train_images=train_images.reshape(len(train_images),28,28,1)
test_images=test_images.reshape(len(test_images),28,28,1)

In [ ]:
def build_model(hp):  
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  return model

In [ ]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  """Entry point for launching an IPython kernel.


In [ ]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=5,directory='output',project_name="Mnist Fashion")

In [ ]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Trial 5 Complete [00h 00m 22s]
val_accuracy: 0.8516666889190674

Best val_accuracy So Far: 0.9118333458900452
Total elapsed time: 00h 02m 43s
INFO:tensorflow:Oracle triggered exit


In [ ]:
model=tuner_search.get_best_models(num_models=1)[0]

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 24, 24, 96)        2496      
                                                                 
 conv2d_1 (Conv2D)           (None, 20, 20, 48)        115248    
                                                                 
 flatten (Flatten)           (None, 19200)             0         
                                                                 
 dense (Dense)               (None, 96)                1843296   
                                                                 
 dense_1 (Dense)             (None, 10)                970       
                                                                 
Total params: 1,962,010
Trainable params: 1,962,010
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(train_images, train_labels, epochs=10, validation_split=0.1, initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.1801 - accuracy: 0.9311 - val_loss: 0.2603 - val_accuracy: 0.9060
Epoch 5/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.1445 - accuracy: 0.9457 - val_loss: 0.2575 - val_accuracy: 0.9095
Epoch 6/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.1136 - accuracy: 0.9577 - val_loss: 0.2824 - val_accuracy: 0.9085
Epoch 7/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0917 - accuracy: 0.9655 - val_loss: 0.3349 - val_accuracy: 0.9017
Epoch 8/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0705 - accuracy: 0.9732 - val_loss: 0.3858 - val_accuracy: 0.9057
Epoch 9/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0582 - accuracy: 0.9784 - val_loss: 0.3567 - val_accuracy: 0.9103
Epoch 10/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0481 - accuracy: 0.9823 - val_loss: 0.4147 - val_accuracy